<a href="https://colab.research.google.com/github/Eiko58/Hippocampus_segmentation/blob/main/balanced_train_folder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader as DataLoader
from torchvision import transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
import glob
from google.colab import drive
import matplotlib.pyplot as plt
import os
import os.path
from pathlib import Path
import cv2
import copy
import torchvision.transforms.functional as TF
import random
import math
import pandas as pd
from torchvision.utils import save_image

In [ ]:
drive.mount('/content/drive/')
!pwd

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content


In [ ]:
path = 'drive/MyDrive/hippocampus/train/label'
index = 0
list_indices = []
for img_name in os.listdir(path):
  print(img_name)
  print(f'{path}/{img_name}')
  img = np.round(cv2.cvtColor(cv2.imread(f'{path}/{img_name}'), cv2.COLOR_BGR2GRAY)/255)
  print(img.sum()==0)
  break
print(img)

ADNI_127_S_0754_23787_hip_135.jpg
drive/MyDrive/hippocampus/train/label/ADNI_127_S_0754_23787_hip_135.jpg
True
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
i = 0
for file in glob.glob(path+'/*.jpg'):
  if i == 1000:
      print(file)
      break
  i+=1
  
i = 0
for file in glob.glob('drive/MyDrive/hippocampus/train/Total/*.jpg'):
  if i == 1000:
      print(file)
      break
  i+=1
  

drive/MyDrive/hippocampus/train/label/ADNI_126_S_0606_37834_hip_080.jpg
drive/MyDrive/hippocampus/train/Total/ADNI_126_S_0606_37834_ACPC_080.jpg


In [ ]:
path = 'drive/MyDrive/hippocampus/train/'
index = -1
list_indices = []
for file in glob.glob(path+'label/*.jpg'):
  index +=1
  img = np.round(cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2GRAY)/255)
  if img.sum() == 0:
    list_indices.append(index)
  


In [ ]:
glob.glob(path+'label/*.jpg')[1:5]

['drive/MyDrive/hippocampus/train/label/ADNI_127_S_0754_23787_hip_136.jpg',
 'drive/MyDrive/hippocampus/train/label/ADNI_127_S_0754_23787_hip_137.jpg',
 'drive/MyDrive/hippocampus/train/label/ADNI_127_S_0754_23787_hip_138.jpg',
 'drive/MyDrive/hippocampus/train/label/ADNI_127_S_0754_23787_hip_139.jpg']

In [ ]:
len(list_indices) #in total 18710 images.
list_indices_nempty = []
for i in range(18710):
  if not i in list_indices:
    list_indices_nempty.append(i)
len(list_indices_nempty)


2969

In [ ]:
random.seed(1234)
indices_keep = random.sample(list_indices, k=3000)
path2 = 'drive/MyDrive/hippocampus/balanced_train'
os.mkdir(path2)
os.mkdir(f'{path2}/Total')
os.mkdir(f'{path2}/label')

FileExistsError: ignored

In [ ]:
indices_keep.extend(list_indices_nempty)

In [ ]:
print(len(indices_keep))
indices_keep = sorted(indices_keep)

In [ ]:

file_list =  glob.glob(path+'label/*.jpg')
files_keep = [file_list[i] for i in indices_keep]
print(len(files_keep))
print(files_keep[1:5])

  

['drive/MyDrive/hippocampus/train/label/ADNI_127_S_0754_23787_hip_145.jpg', 'drive/MyDrive/hippocampus/train/label/ADNI_127_S_0754_23787_hip_151.jpg', 'drive/MyDrive/hippocampus/train/label/ADNI_127_S_0754_23787_hip_159.jpg', 'drive/MyDrive/hippocampus/train/label/ADNI_127_S_0754_23787_hip_162.jpg']


In [ ]:
file_list_total =  glob.glob(path+'Total/*.jpg')
files_keep_total = [file_list_total[i] for i in indices_keep]
print(len(files_keep_total))
print(files_keep_total[1:5])

5969
['drive/MyDrive/hippocampus/train/Total/ADNI_127_S_0754_23787_ACPC_145.jpg', 'drive/MyDrive/hippocampus/train/Total/ADNI_127_S_0754_23787_ACPC_151.jpg', 'drive/MyDrive/hippocampus/train/Total/ADNI_127_S_0754_23787_ACPC_159.jpg', 'drive/MyDrive/hippocampus/train/Total/ADNI_127_S_0754_23787_ACPC_162.jpg']


In [ ]:
import shutil
for file in files_keep:
  new_path = file.replace('train', 'balanced_train')
  shutil.copy(file, new_path)

In [ ]:
print(len(glob.glob('drive/MyDrive/hippocampus/balanced_train/label/*.jpg')))
print(glob.glob('drive/MyDrive/hippocampus/balanced_train/label/*.jpg')[1:5])

5969
['drive/MyDrive/hippocampus/balanced_train/label/ADNI_127_S_0754_23787_hip_145.jpg', 'drive/MyDrive/hippocampus/balanced_train/label/ADNI_127_S_0754_23787_hip_151.jpg', 'drive/MyDrive/hippocampus/balanced_train/label/ADNI_127_S_0754_23787_hip_159.jpg', 'drive/MyDrive/hippocampus/balanced_train/label/ADNI_127_S_0754_23787_hip_162.jpg']


In [ ]:
for file in files_keep_total:
  new_path = file.replace('train', 'balanced_train')
  shutil.copy(file, new_path)

In [ ]:
print(len(glob.glob('drive/MyDrive/hippocampus/balanced_train/Total/*.jpg')))
print(glob.glob('drive/MyDrive/hippocampus/balanced_train/Total/*.jpg')[1:5])

5969
['drive/MyDrive/hippocampus/balanced_train/Total/ADNI_127_S_0754_23787_ACPC_145.jpg', 'drive/MyDrive/hippocampus/balanced_train/Total/ADNI_127_S_0754_23787_ACPC_151.jpg', 'drive/MyDrive/hippocampus/balanced_train/Total/ADNI_127_S_0754_23787_ACPC_159.jpg', 'drive/MyDrive/hippocampus/balanced_train/Total/ADNI_127_S_0754_23787_ACPC_162.jpg']


In [ ]:
print(glob.glob('drive/MyDrive/hippocampus/balanced_train/Total/*.jpg')[3000])
print(glob.glob('drive/MyDrive/hippocampus/balanced_train/label/*.jpg')[3000])

drive/MyDrive/hippocampus/balanced_train/Total/ADNI_018_S_0425_108829_ACPC_027.jpg
drive/MyDrive/hippocampus/balanced_train/label/ADNI_018_S_0425_108829_hip_027.jpg
